In [ ]:
import opensmile
import pandas as pd
from glob import glob

In [ ]:
def feat_compare(filePaths, csvName=None):
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.Functionals,
        # feature_level: Functionals, LowLevelDescriptors, LowLevelDescriptors_Deltas
    )
    df = pd.DataFrame()
    for fn in filePaths:
        df = df.append(smile.process_file(fn))
    if csvName is not None:
        df.to_csv(csvName, index=False)
    return df

In [ ]:
data_path = '../ADReSS-IS2020-data'
if 0:
    df_test = feat_compare(sorted(glob(data_path+'/test/par/*.wav')))
    df_train_cc = feat_compare(sorted(glob(data_path+'/train/par/cc/*.wav')))
    df_train_cd = feat_compare(sorted(glob(data_path+'/train/par/cd/*.wav')))
    df_test.to_csv('feats/ComPaRE_tst.csv')
    df_train_cc.to_csv('feats/ComPaRE_cc.csv')
    df_train_cd.to_csv('feats/ComPaRE_cd.csv')

In [ ]:
df_train_cc = pd.read_csv('feats/compare_cc.csv')
df_train_cd = pd.read_csv('feats/compare_cd.csv')
df_train = pd.concat([df_train_cc, df_train_cd], ignore_index=True)
df_train.drop(['start','end'], axis=1, inplace=True)
df_train = df_train.rename(columns={'file': 'id'})
df_train['id'] = df_train['id'].apply(lambda x: x[-8:-4])

df_test = pd.read_csv('feats/compare_tst.csv')
df_test.drop(['start','end'], axis=1, inplace=True)
df_test = df_test.rename(columns={'file': 'id'})
df_test['id'] = df_test['id'].apply(lambda x: x[-8:-4])

chas_trn = pd.read_csv('feats/chas_trn.csv', usecols=['id', 'sex', 'age', 'ad'])
chas_tst = pd.read_csv('feats/chas_tst.csv', usecols=['id', 'sex', 'age'])

In [ ]:
chas_trn = chas_trn.sort_values(by=['id']).reset_index(drop=True)
chas_tst = chas_tst.sort_values(by=['id']).reset_index(drop=True)
compare_trn = pd.merge(chas_trn, df_train, on="id")
compare_tst = pd.merge(chas_tst, df_test, on="id")
compare_trn.to_csv('feats/compare_par_trn.csv', index=False)
compare_tst.to_csv('feats/compare_par_tst.csv', index=False)